In [2]:
import pandas as pd
import numpy as np
import openpyxl, config
import os, time, re
from dailyreporting import *

In [10]:
datestart = '2022-04-13'
dateend = '2022-04-14'
unitname = 'TAA1'

t0 = time.time()
print('Date:')
print(f"{datestart} to {dateend}")
print(type(datestart), type(dateend))

try: 
    datestart = pd.to_datetime(datestart).strftime('%Y-%m-%d %X')
    dateend = pd.to_datetime(dateend).strftime('%Y-%m-%d %X')
except: 
    datestart = (pd.to_datetime(time.ctime()).strftime('%Y-%m-%d %X') - pd.to_timedelta(24, 'h')).strftime('%Y-%m-%d %X')
    dateend = pd.to_datetime(time.ctime()).strftime('%Y-%m-%d %X')

Date:
2022-04-13 to 2022-04-14
<class 'str'> <class 'str'>


In [11]:
datestart = pd.to_datetime(datestart).strftime('%Y-%m-%d')
dateend = pd.to_datetime(dateend).strftime('%Y-%m-%d')

db_config = config.UNIT_CONFIG[unitname]
con = f'mysql+mysqlconnector://root:P%40ssw0rd@{db_config["HOST"]}/{db_config["DB"]}'

TAG_ENABLE_COPT = db_config['TAG_ENABLE_COPT']
TAG_ENABLE_SOPT = db_config['TAG_ENABLE_SOPT']
starttime, endtime = ['','']

# Boiler Auto Tuning Enable Status
print('Get historical status ...')
q = f"""SELECT f_address_no, f_date_rec, f_value FROM tb_bat_history 
    WHERE f_address_no IN ("{TAG_ENABLE_COPT}","{TAG_ENABLE_SOPT}")
    AND f_date_rec BETWEEN "{datestart} {starttime}" AND "{dateend} {endtime}" """

df = pd.read_sql(q, con)
df['f_value'] = df['f_value'].astype(float)
Enable_status = df.pivot('f_date_rec', 'f_address_no', 'f_value')
Enable_status['BAT'] = Enable_status.max(axis=1)

Total_minutes = len(Enable_status.resample('1min').mean())
Enable_status_sum = Enable_status.sum()
Enable_status_sum

Get historical status ...


f_address_no
EWS102/10CH:SELENABLE.BO01       1123.0
EWS102/10FDF:SELENABLE_M.BO01       0.0
BAT                              1123.0
dtype: float64

In [12]:
# SOPT Impossible Condition to Enable
print('Get historical SOPT safeguards ...')
q = f"""SELECT dtl.f_sequence, raw.f_date_rec, dtl.f_bracket_open, dtl.f_tag_sensor, 
    conf.f_description, raw.f_value, dtl.f_bracket_close
    FROM tb_sootblow_rules_hdr head
    LEFT JOIN tb_sootblow_rules_dtl dtl 
    ON head.f_rule_hdr_id = dtl.f_rule_hdr_id
    LEFT JOIN tb_tags_read_conf conf
    ON dtl.f_tag_sensor = conf.f_tag_name 
    LEFT JOIN tb_bat_history raw 
    ON dtl.f_tag_sensor = raw.f_address_no
    WHERE head.f_rule_descr = "Safeguard"
    AND raw.f_date_rec BETWEEN "{datestart} {starttime}" AND "{dateend} {endtime}" """
    # TODO: Waiting for watchdog fixing
    # UNION
    # SELECT 0 AS f_sequence, f_date_rec, '(' as f_bracket_open, f_address_no , f_address_no AS f_description, f_value , '=1)' AS f_bracket_close 
    # FROM tb_bat_history
    # WHERE f_address_no = "WatchdogStatus"
    # AND f_date_rec BETWEEN "{datestart} {starttime}" AND "{dateend} {endtime}" """
df = pd.read_sql(q, con)
df['f_value'] = df['f_value'].astype(float)
df['rule'] = df['f_bracket_open'] + df['f_value'].astype(str) + df['f_bracket_close'].str.replace(" AND","")
df['result'] = [eval(f.replace('=','==')) for f in df['rule']]
df['equation'] = df['f_bracket_open'] + df['f_description'].astype(str) + df['f_bracket_close'].str.replace(" AND","")
df['equation'] = df['equation'].str.slice(1,-1)

Safeguard_SOPT = df.pivot_table(index='f_date_rec', columns='equation', values='result')
Safeguard_SOPT['Safeguard SOPT'] = Safeguard_SOPT.min(axis=1)

Safeguard_sum_SOPT = pd.DataFrame(Safeguard_SOPT.sum(), columns=['Enabled'])
Safeguard_sum_SOPT['Disabled'] = Total_minutes - Safeguard_sum_SOPT['Enabled']
Safeguard_sum_SOPT

Get historical SOPT safeguards ...


,Enabled,Disabled
equation,,
BOILER DRUM LEVEL 3<120,1441,0
BOILER MFT TURB TRIP(SOE)=0,1441,0
FURNACE PRESS #1< -10,1438,3
GEN ACTIVE POWER>240,1441,0
Safeguard SOPT,1438,3


In [13]:
# COPT Impossible Condition to Enable
print('Get historical COPT safeguards ...')
q = f"""SELECT dtl.f_sequence, raw.f_date_rec, dtl.f_bracket_open, dtl.f_tag_sensor, 
        conf.f_description, raw.f_value, dtl.f_bracket_close
        FROM tb_combustion_rules_hdr head
        LEFT JOIN tb_combustion_rules_dtl dtl 
        ON head.f_rule_hdr_id = dtl.f_rule_hdr_id
        LEFT JOIN tb_tags_read_conf conf
        ON dtl.f_tag_sensor = conf.f_tag_name 
        LEFT JOIN tb_bat_history raw 
        ON dtl.f_tag_sensor = raw.f_address_no
        WHERE head.f_rule_descr = "SAFEGUARD"
        AND raw.f_date_rec BETWEEN "{datestart} {starttime}" AND "{dateend} {endtime}" """
        # TODO: Waiting for watchdog fixing
        # UNION
        # SELECT 0 AS f_sequence, f_date_rec, '(' as f_bracket_open, f_address_no , f_address_no AS f_description, f_value , '=1)' AS f_bracket_close 
        # FROM tb_bat_history
        # WHERE f_address_no = "WatchdogStatus"
        # AND f_date_rec BETWEEN "{datestart} {starttime}" AND "{dateend} {endtime}" """
df = pd.read_sql(q, con)
df['f_value'] = df['f_value'].astype(float)
df['rule'] = df['f_bracket_open'] + df['f_value'].astype(str) + df['f_bracket_close'].str.replace(" AND","")
df['result'] = [eval(f.replace('=','==')) for f in df['rule']]
df['equation'] = df['f_bracket_open'] + df['f_description'].astype(str) + df['f_bracket_close'].str.replace(" AND","")
df['equation'] = df['equation'].str.slice(1,-1)

Safeguard_COPT = df.pivot_table(index='f_date_rec', columns='equation', values='result')
Safeguard_COPT['Safeguard COPT'] = Safeguard_COPT.min(axis=1)

Safeguard_sum_COPT = pd.DataFrame(Safeguard_COPT.sum(), columns=['Enabled'])
Safeguard_sum_COPT['Disabled'] = Total_minutes - Safeguard_sum_COPT['Enabled']
Safeguard_sum_COPT

Get historical COPT safeguards ...


,Enabled,Disabled
equation,,
BOILER DRUM LEVEL 3<120,1441,0
BOILER MFT TURB TRIP(SOE)=0,1441,0
FEGT<1300,1441,0
FURNACE PRESS #1< -10,1438,3
GEN ACTIVE POWER>240,1441,0
No.1 FD FAN MOVABLE BLADE POSI> 5,1441,0
No.1 ID FAN FIXED BLADE POSI<97,1441,0
No.1 ID FAN MOT CURRENT<205,1441,0
No.1 PA FAN MOT CURRENT<205,1441,0


In [14]:
Safeguard = Safeguard_COPT.merge(Safeguard_SOPT.astype(float), how='left', left_index=True, right_index=True)
Safeguard_sum = pd.DataFrame(Safeguard.sum(), columns=['Enabled'])
Safeguard_sum['Disabled'] = Total_minutes - Safeguard_sum['Enabled']
Safeguard_sum

,Enabled,Disabled
equation,,
BOILER DRUM LEVEL 3<120_x,1441.0,0.0
BOILER MFT TURB TRIP(SOE)=0_x,1441.0,0.0
FEGT<1300,1441.0,0.0
FURNACE PRESS #1< -10_x,1438.0,3.0
GEN ACTIVE POWER>240_x,1441.0,0.0
No.1 FD FAN MOVABLE BLADE POSI> 5,1441.0,0.0
No.1 ID FAN FIXED BLADE POSI<97,1441.0,0.0
No.1 ID FAN MOT CURRENT<205,1441.0,0.0
No.1 PA FAN MOT CURRENT<205,1441.0,0.0


In [15]:
# Total Possible Condition Time
print('Calculating possible condition to enabled time')
Possible_df = Safeguard_sum.loc[['Safeguard SOPT', 'Safeguard COPT']]
BAT_possible_df = pd.DataFrame(Possible_df.iloc[np.argmax(Possible_df['Enabled'])]).T
BAT_possible_df.index = ['BAT']

Possible_df = Possible_df.append(BAT_possible_df)
Possible_df

Calculating possible condition to enabled time


,Enabled,Disabled
Safeguard SOPT,1438.0,3.0
Safeguard COPT,1438.0,3.0
BAT,1438.0,3.0


In [17]:
# Percentage of BAT Enabled
Enabled_status_df = pd.DataFrame(Enable_status_sum, columns=['Enabled'])
Enabled_status_df = Enabled_status_df.rename(index={TAG_ENABLE_SOPT:'Safeguard SOPT', TAG_ENABLE_COPT:'Safeguard COPT'})
Enabled_status_df = Enabled_status_df.merge(Possible_df[['Enabled']].rename(columns={'Enabled':'Total'}), how='left', left_index=True, right_index=True)
Enabled_status_df['Disabled'] = Enabled_status_df['Total'] - Enabled_status_df['Enabled']
Enabled_status_df[['Enabled','Disabled','Total']]

,Enabled,Disabled,Total
f_address_no,,,
Safeguard SOPT,1123.0,315.0,1438.0
Safeguard COPT,0.0,1438.0,1438.0
BAT,1123.0,315.0,1438.0
